In [1]:
from yasearch.etc.schema import Document
import numpy as np
from loguru import logger
from tqdm.autonotebook import tqdm
from pathlib import Path
import os
import polars as pl
import uuid
from pathlib import Path
import simplejson as json
from typing import Optional, List, Dict
from weaviate.classes.query import MetadataQuery

/var/folders/nn/jz242fbn1kn9tk10ncm2sgzh0000gn/T/ipykernel_69939/3945488611.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
!pip freeze | grep weaviate

weaviate-client==4.5.4


In [3]:
from transformers import AutoModel, AutoTokenizer
from torch import Tensor
import torch
from torch.functional import F
from functools import partial
from more_itertools import chunked
from yasearch.processing import loader, igniset

In [4]:
model_name_or_path = "intfloat/multilingual-e5-base"

In [5]:
model = AutoModel.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [6]:
def prefix(x, pref:str):
    return pref.strip() + " " + x

In [7]:
def ignite_dataset(where) -> List[Dict]:
    docs = None
    with open(str(Path(where))) as fin:
        docs = json.load(fin)
    return docs

In [8]:
docs = ignite_dataset(where=Path(os.getcwd()) / ".data" / "polaroids.ai.data.json")

In [9]:
pl_data = pl.from_dicts(docs)

In [10]:
pl_data.head()

content,title,author,type,has_image,img_path,speaker,query
str,str,str,str,bool,str,str,str
"""В реалисте вер…","""Братья Карамаз…","""Ф.М. Достоевск…","""book""",false,null,null,null
"""Жизнь — это ми…","""Человек в футл…","""Антон Павлович…","""book""",false,null,null,null
"""Нет, не так. К…","""Метро 2033""","""Дмитрий Глухов…","""book""",true,"""./img/metro203…",null,null
"""Станьте солнце…","""Преступление и…","""Ф.М.Достоевски…","""book""",true,"""./img/crimeand…",null,null
"""Когда-то он бы…","""Джон Уик 3""","""...""","""movie""",true,"""./img/johnwick…",null,null


In [11]:
def tokenize(input_texts, tokenizer):
    batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
    return batch_dict

In [12]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [13]:
documents = [d.get("content") for d in docs]
labels = [d.get("title") for d in docs]

In [14]:
tokenize(documents, tokenizer=tokenizer)["input_ids"].shape

torch.Size([616, 512])

In [15]:
def generate_unique_id(text, namespace_uuid='91461c99-f89d-49d2-af96-d8e2e14e9b58'):
    """
    
    Args:
    text (str): Textual content for generating fixed UUID.
    namespace_uuid (str): Namespace used by https://rethinkdb.com.
    
    Returns:
    str: Строковое представление детерминированного UUID.
    """
    namespace = uuid.UUID(namespace_uuid)
    
    deterministic_uuid = uuid.uuid5(namespace, text)
    
    return str(deterministic_uuid)

In [16]:
def vectorize(docs, labels: Optional[List[str]] = None, pref:str="", norm: bool = True, batch_size:int = 2):
    wrapped_docs = []
    if labels is not None:
        it = chunked(zip(docs, labels), n=batch_size)
    else:
        it = chunked(docs, n=batch_size)
    pbar = tqdm(total=len(docs), desc=" Embeddings for documents")
    for chunk in it:
        if labels is not None:
            raw_docs, raw_labels = [c[0] for c in chunk], [c[1] for c in chunk]
        else:
            raw_docs = chunk
        _docs = [prefix(x, pref=pref) for x in raw_docs]
        batch_dict = tokenize(_docs, tokenizer=tokenizer)
        with torch.no_grad():
            outputs = model(**batch_dict)
            _embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask']).cpu()
        if norm:
            _embeddings = F.normalize(_embeddings, p=2, dim=len(_embeddings.shape) - 1)
        _embeddings = _embeddings.numpy()
        if labels is not None:
            chunk = [Document.from_dict({"content": doc, "embedding": list(emb), "labels": [generate_unique_id(label)]}) for doc, label, emb in zip(raw_docs, raw_labels, _embeddings)]
        else:
            chunk = [Document.from_dict({"content": doc, "embedding": list(emb)}) for doc, emb in zip(raw_docs, _embeddings)]
        wrapped_docs.extend(chunk)
        pbar.update(n=len(raw_docs))
    return wrapped_docs

In [17]:
from yasearch.storing.weaviate import WeaviateDocStore
store = WeaviateDocStore(url="http://localhost:2211", collection_name="justatom")

In [18]:
store.count_documents()

84

In [19]:
wrapped_docs = vectorize(docs=documents, labels=labels, pref="passage: ")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
 Embeddings for documents: 100%|██████████| 616/616 [00:49<00:00, 12.49it/s]


In [20]:
wrapped_docs[4].content

'Когда-то он был в нашей команде. Его прозвали “Баба-Яга”. Джон - человек целеустремленный, обязательный, волевой. Тебе все это знакомо очень мало... Я видел однажды, как он убил троих в баре... Карандашом. Обычным карандашом. и, вдруг, в один прекрасный день, он захотел уйти. Все из-за женщины, конечно. Поэтому, я заключил с ним сделку. Я дал ему невыполнимое задание - работу, которую никто не смог потянуть. Те, которых он закопал тогда легли в основу нашей организации.\nА теперь мой сын через несколько дней после смерти его жены стырил его тачку и убил его щенка...\nЙозеф, Йозеф, мой сын. Джон придет за тобой и ты ничего не сможешь сделать потому что не сможешь.'

In [21]:
assert wrapped_docs[0].meta["labels"][0] == generate_unique_id(labels[0]), f"Unexptected error due to labeling errors"

In [22]:
wrapped_docs[0].meta

{'labels': ['efc12637-0e91-581d-bdba-64fba3ec6a04']}

In [38]:
store.write_documents(documents=wrapped_docs)

616

In [39]:
queries = [
    "Какие правила голодных игр?",
    "Баба Яга",
    "Любовь"
]

In [40]:
query_vecs = vectorize(queries, pref="query: ")

 Embeddings for documents: 100%|██████████| 3/3 [00:00<00:00, 30.21it/s]


In [41]:
print(query_vecs[0].content)
print(query_vecs[0].embedding)

Какие правила голодных игр?
[0.05304961, 0.049881455, 0.006537094, 0.05279775, 0.049019434, -0.025354123, -0.016592989, -0.058351066, 0.043995507, 0.029732578, -0.0053635505, 0.028418316, 0.17047882, 0.037980255, -0.046131313, -0.053106647, 0.028046364, 0.006709769, 0.017089305, 0.026183996, 0.016328951, -0.03578681, 0.017885763, -0.0063233525, 0.025755985, -0.036064174, 0.009998725, 0.03402224, -0.023247937, 0.024673829, 0.030762691, -0.054473262, -0.00026232912, 0.021900875, 0.0240265, 0.021065347, 0.002189402, -0.016787646, 0.002853894, 0.025357524, -0.012233664, -0.013349867, 0.054261386, -0.06845884, 0.008962846, -0.019977212, 0.03098431, -0.0045971773, -0.055024866, -0.047476973, 0.0017279275, 0.025206815, 0.04284553, 0.021489918, -0.05621173, -0.06356004, 0.03876085, 0.028062899, -0.053602245, 0.03184615, 0.0027324294, 0.056962892, -0.015343961, 0.05477506, 0.044966616, -0.018142343, 0.032350183, -0.042801116, -0.04087332, -0.004499426, -0.014831031, -0.0006840422, 0.039676137, 

#### Seach `by embedding` only

In [43]:
store.search_by_embedding(query_embedding=query_vecs[0].embedding, top_k=2)

[<Document: {'content': 'Проигрывать всегда горько. Неистово верить в свою правоту, и всё равно проиграть.', 'content_type': 'text', 'score': 0.90164715051651, 'meta': {'labels': [UUID('f5bced10-6b44-5ba4-bbc0-93ce7cf23f4e')], 'dataframe': None}, 'embedding': '<embedding of shape [no shape]>', 'id': '37c3fd8d7296de6e7f60f63073044af2'}>,
 <Document: {'content': 'В Дистрикте-12 голодная смерть не редкость. За примерами далеко ходить не надо: старики, не способные больше работать, дети из семей, где слишком много ртов, рабочие, искалеченные в шахтах. Бродил вчера человек по улицам, а сегодня, смотришь, лежит где-нибудь, привалившись к забору, и не шевелится. Или на Луговине наткнешься. А другой раз только плач из домов слышишь. Приедут миротворцы, заберут тело. Власти не признают, что это из-за голода. Официально причина всегда – грипп, переохлаждение или воспаление легких.', 'content_type': 'text', 'score': 0.9013684988021851, 'meta': {'labels': [UUID('721626b0-f98a-55d8-b782-9dd6c2185bd

#### Search `by bm25` only

In [45]:
store.search_by_keywords(query=queries[0], top_k=2)

[<Document: {'content': 'Герой наш поворотился в ту ж минуту к губернаторше и уже готов был отпустить ей ответ, вероятно ничем не хуже тех, какие отпускают в модных повестях Звонские, Линские, Лидины, Гремины и всякие ловкие военные люди, как, невзначай поднявши глаза, остановился вдруг, будто оглушенный ударом.', 'content_type': 'text', 'score': 1.8574644327163696, 'meta': {'labels': [UUID('f9507682-c01d-5f7b-8cb4-437ba795d921')], 'dataframe': None}, 'embedding': None, 'id': 'b3d5e98f550b97faed8dca802838910e'}>,
 <Document: {'content': '– Будут какие-нибудь советы? – интересуется Пит.\n– Как только ударят в гонг, скорее уносите ноги. Мясорубка перед Рогом изобилия вам не по зубам. Улепетывайте что есть духу, чем дальше от других, тем лучше, и ищите источник воды. Ясно?\n– А потом? – спрашиваю я.\n– А потом постарайтесь выжить, – отвечает Хеймитч.', 'content_type': 'text', 'score': 1.8173242807388306, 'meta': {'labels': [UUID('721626b0-f98a-55d8-b782-9dd6c2185bd6')], 'dataframe': None}

#### Search by both `bm25` and `embedding` using `alpha` score to merge the rankings from both

In [52]:
alpha = 0.6
query = query_vecs[0].content
query_emb = query_vecs[0].embedding

response = store.search(query=query, query_embedding=query_emb, alpha=alpha, top_k=2)

response_docs = "\n---".join([f"\nDOC[{str(pos)}]\n" + x.content for pos, x in enumerate(response)])
logger.info(f" >> | {query}")
logger.info(response_docs)

2024-06-14 21:40:26.890 | INFO     | __main__:<module>:8 -  >> | Какие правила голодных игр?
2024-06-14 21:40:26.891 | INFO     | __main__:<module>:9 - 
DOC[0]
– Будут какие-нибудь советы? – интересуется Пит.
– Как только ударят в гонг, скорее уносите ноги. Мясорубка перед Рогом изобилия вам не по зубам. Улепетывайте что есть духу, чем дальше от других, тем лучше, и ищите источник воды. Ясно?
– А потом? – спрашиваю я.
– А потом постарайтесь выжить, – отвечает Хеймитч.
---
DOC[1]
Проигрывать всегда горько. Неистово верить в свою правоту, и всё равно проиграть.
